In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import re
import nltk
#import wandb
import torch
import random
import string
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import cm
from tensorflow import keras
from textblob import TextBlob
from keras import regularizers
from string import punctuation
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
from keras.models import Sequential
from tqdm.keras import TqdmCallback
from keras.utils import to_categorical
from transformers import EvalPrediction
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
from transformers import TFDistilBertModel
from keras.layers import BatchNormalization
from transformers import DistilBertTokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from keras.preprocessing.text import Tokenizer
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import GRU, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from transformers import AutoTokenizer, TFDistilBertForSequenceClassification
from keras.layers import Input, Dense, Embedding, Dropout, Reshape, Flatten, LSTM, Bidirectional
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
df = pd.read_csv('labels.csv')
df = df[['text_corrected','overall_sentiment']]

In [5]:
df['overall_sentiment'] = df['overall_sentiment'].replace(['very_positive', 'positive',
                                                     'neutral', 'negative', 
                                                     'very_negative'], 
                                                    [2, 2, 1, 0, 0])

In [6]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [7]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-o7_po07o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-o7_po07o
  Resolved https://github.com/huggingface/accelerate to commit 86720fdb11165348ed6b575a56d0cb597b8554f4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
bert = 'ahmedrachid/FinancialBERT-Sentiment-Analysis'

tokenizer = BertTokenizer.from_pretrained(bert)
model = BertForSequenceClassification.from_pretrained(bert, problem_type="multi_label_classification", num_labels=3)

In [10]:
X, y  = df['text_corrected'], df['overall_sentiment']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2)

In [12]:
train_df = pd.DataFrame({'text': X_train, 'label': y_train})
train_labels = train_df['label'].values
test_df = pd.DataFrame({'text': X_test, 'label': y_test})
test_labels = test_df['label'].values
val_df = pd.DataFrame({'text': X_val, 'label': y_val})
val_labels = val_df['label'].values

In [13]:
tokens_train = tokenizer.batch_encode_plus(train_df['text'].values, padding='max_length',
                                           max_length=512, truncation=True)
tokens_test = tokenizer.batch_encode_plus(test_df['text'].values, padding='max_length', 
                                          max_length=512, truncation=True)
tokens_val = tokenizer.batch_encode_plus(val_df['text'].values, padding='max_length',
                                         max_length=512, truncation=True)

In [14]:
class Dataset(torch.utils.data.Dataset):
    # создание пользовательского датасета
    def __init__(self, encodings, labels, unique_labels):
        self.encodings = encodings
        self.labels = labels
        self.unique_labels = unique_labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        labels = self.unique_labels * [0]
        labels[int(self.labels[idx])] = 1
        item["overall_sentiment"] = torch.tensor(labels).float()
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(tokens_train, train_labels, 3)
test_dataset = Dataset(tokens_test, test_labels, 3)
val_dataset = Dataset(tokens_val, val_labels, 3)

In [15]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'f1_score' : f1_score(predictions, labels, average='micro')}

In [ ]:
args = TrainingArguments(output_dir='temp/', evaluation_strategy='epoch',
        save_strategy='epoch', learning_rate=2e-5,
        per_device_train_batch_size=8, per_device_eval_batch_size=8,
        num_train_epochs=5, weight_decay=0.01, load_best_model_at_end=True,
        metric_for_best_model='f1_score',
)

trainer = Trainer(model=bert, args=args, train_dataset=train_df, 
                  eval_dataset=val_df, compute_metrics=compute_metrics)

trainer.train()